In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)
print(tf.config.experimental.list_physical_devices('GPU'))




2.10.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
tf.__version__

'2.10.0'

In [3]:

#For Data Augmentation Porpuses 
myDataGen_train=ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 80,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.3,
                                   horizontal_flip = True 
                                   )
myDataGen_test=ImageDataGenerator(rescale = 1./255.)

In [4]:
import os

In [5]:

training_dir=os.path.join(os.getcwd(),"./archive/train")
test_dir=os.path.join(os.getcwd(),"./archive/test")
print(os.listdir(training_dir))
print(os.listdir(test_dir))


['18-20', '21-30', '31-40', '41-50', '51-60']
['18-20', '21-30', '31-40', '41-50', '51-60']


In [6]:
train_ds=myDataGen_train.flow_from_directory(training_dir,class_mode="categorical",target_size=(400,400))
test_ds=myDataGen_test.flow_from_directory(test_dir,class_mode="categorical",target_size=(400,400))


Found 125 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [7]:
from tensorflow.keras.applications import InceptionV3
#Definition of The Model Using
#In this case we will use the Inception V3 Model

pre_trained_model=InceptionV3(input_shape=(400,400,3),
                            include_top=False,
                            weights=None)

In [8]:
model_weights=os.path.join(os.getcwd(),"./weightsNN","inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5")

pre_trained_model.load_weights(model_weights)


In [9]:
#Lest dive into the model a little bit
for layer in pre_trained_model.layers:
    #print(layer.trainable)
    layer.trainable=False

pre_trained_model.summary()



Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 400, 400, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 199, 199, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 199, 199, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [10]:
last_usable_layer=pre_trained_model.get_layer("mixed7")

last_output=last_usable_layer.output

In [11]:
from keras.layers import Flatten,Dense,Dropout
from keras import Model
from keras.optimizers import Adam

x=Flatten()(last_output)
x=Dense(1024,activation="relu")(x)
x=Dropout(0.2)(x)
x=Dense(1024,activation="relu")(x)
x=Dense(1024,activation="relu")(x)
x=Dense(5,activation="softmax")(x)

final_model=Model(pre_trained_model.input,x)
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 400, 400, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 199, 199, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 199, 199, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [12]:
final_model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


ValueError: Unknown optimizer: sdg. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
!pip list 

Package                      Version
---------------------------- ----------
absl-py                      2.0.0
appnope                      0.1.3
asttokens                    2.4.1
astunparse                   1.6.3
cachetools                   5.3.2
certifi                      2023.11.17
charset-normalizer           3.3.2
comm                         0.2.0
debugpy                      1.8.0
decorator                    5.1.1
exceptiongroup               1.2.0
executing                    2.0.1
flatbuffers                  23.5.26
gast                         0.5.4
google-auth                  2.23.4
google-auth-oauthlib         1.1.0
google-pasta                 0.2.0
grpcio                       1.59.3
h5py                         3.10.0
idna                         3.6
ipykernel                    6.26.0
ipython                      8.18.0
jedi                         0.19.1
jupyter_client               8.6.0
jupyter_core                 5.5.0
keras                        2.15.0
l

In [ ]:
import sys
import PIL as pl
import scipy
print(pl.__version__)
history=final_model.fit(train_ds,validation_data=test_ds,epochs=20,verbose=2)

8.4.0
Epoch 1/20


ResourceExhaustedError: Graph execution error:

Detected at node 'model/batch_normalization_96/FusedBatchNormV3' defined at (most recent call last):
    File "c:\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\aeaqu\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Python310\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "C:\Users\aeaqu\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Python310\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Python310\lib\asyncio\base_events.py", line 595, in run_forever
      self._run_once()